# CCD Basics I  Photometric data
________________________________________________________________________________________-

This exercise is adapted from an introductory tutorial to IRAF (Image Reduction and Analysis Facility): http://iraf.noao.edu/ 

The IRAF tools are now no longer supported by the National Optical Astronomy Observatory, but most have been imported into equivalent python routines within the Astropy package. Right now, however, we're just using basic image math to apply the necessary corrections to data frames.

This exercise is designed to show you how to accomplish preliminary reductions of CCD data, including:
 - trimming of the overscan region
 - subtraction of the bias or zero level, and 
 - flat field correction.

The images for this exercise are direct imaging data taken at the Kitt Peak National Observatory by Dr. George Jacoby. The files are called m92001.fits, m92006.fits, etc.

These image files are the data and calibration frames for some images of M92. There is one master bias or "zero" frame and also two flat field frames and four images all taken through either B or V filters.

We will work through spectroscopic data reductions (files: sp\*.fits) in another excercise.

This exercise assumes that you have:
(1) installed Astroconda on your laptop and 
(2) worked through the previous homework assignments and have some familiarity with the basics of python.


In [ ]:
# initial imports of python packages
import numpy as np
import matplotlib.pyplot as plt

# change some default plotting parameters
import matplotlib as mpl
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.interpolation'] = 'nearest'
mpl.rcParams['image.cmap'] = 'Greys_r'

# run the %matplotlib magic command to enable inline plotting
# in the current Notebook
%matplotlib inline

## Handling FITS files

For more information about astropy.io.fits, see http://docs.astropy.org/en/stable/io/fits/index.html

In [ ]:
#import astropy fits file handling package and read data array from fits file
from astropy.io import fits
sci_fn = 'm92001.fits'
sci_hdulist = fits.open(sci_fn)

In [ ]:
#give fits info
sci_hdulist.info()

In [ ]:
# define the data array and list size and data type
data = sci_hdulist[0].data.astype(np.float)
print(data.shape)
print(data.dtype)

## FITS headers

In [ ]:
# extract the data header and print it
hdr = sci_hdulist[0].header
hdr

**QUESTION:** This file is an average of several bias frames. Investigate the header information above to find out how many frames were averaged to create this file. 

For the data reductions for your observing project, you will create a master bias like this one from the bias frames you take at the 16-inch telescope. 

Let's explore the header information a bit more now. From the output above, we see that the type of data file is listed in the "IMAGETYP" keyword of the header.

We can also get header information by keyword indexing:

In [ ]:
hdr['IMAGETYP']

Another way to get the image header:

In [ ]:
from astropy.io.fits import getheader
hdr = getheader('m92001.fits')                              
val = hdr[10]                       # get the 11th keyword's value *remember 0 indexing!
val



**QUESTION:** The value of the 10th keyword is ____? What is the name of 10th header keyword? (Compare with the full header above)

**QUESTION:** Add a cell below to get the DATE of the observation.

Astropy installs some useful scripts you can use on the command line (outside of this notebook) to inspect the headers of your FITS files: http://docs.astropy.org/en/stable/io/fits/usage/scripts.html

In a terminal window running astroconda (i.e. after typing "source activate astroconda3") use the fitsheader tool to look at the IMAGETYP keyword of all the m92\*.fits files:
 
 > fitsheader --keyword IMAGETYP m92\*.fits

**QUESTION:** Copy and paste the output of the fitsheader command above. Now we know the values of the IMAGETYP for each image in the M92 observation set in your data directory


## Image display


In [ ]:
# display the data file 
from astropy.visualization import (MinMaxInterval, SqrtStretch, ImageNormalize)
norm = ImageNormalize(data, interval=MinMaxInterval(),stretch=SqrtStretch())
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
im = ax.imshow(data, origin='lower', norm=norm)
fig.colorbar(im)
#plt.imshow(data, scale='sqrt', percent=99.5)

In [ ]:
#plot a histogram of data values
plt.hist(data.ravel(), bins=20, range=(0,800))
plt.xlabel("Pixel value")
plt.ylabel("No. pixels")
plt.show

**QUESTION:** Comment on the appearance of the greyscale image of the data frame you made above. (Recall the image type!)

**QUESTION:** How does the histogram relate to the appearance of the greyscale image?


## Overscan trimming

We need to find the location of the overscan region on the CCD chip, which determines the trimming parameters and the output image size. For this chip the overscan region is the last several columns. The overscan region and any bad rows or columns along the edges of the frame are then trimmed from the image to produce a trimmed output image (still unreduced). 

We determine these parameters by examining an image OTHER THAN the master bias frame in ds9. A flat field works best.
*Note: Under the Scale menu, uncheck "Use DATASEC" if it is checked. Also, it may help here to adjust the scale to 99% or similar using the top menu bar.*

Use the ds9 window to determine the overscan region, i.e. the columns where the count values drop to a level that is approximately the level of the master bias frame you've been looking at up to this point.

**QUESTION:** What columns are these overscan columms?

**QUESTION:** How does your answer to the previous question relate to the DATASEC and BIASSEC parameters in the header information above? What is the convention for specifying the columns & rows? 

**QUESTION:** In an xterm window running astroconda3, change directories to the data directory where the m92\*.fits files are found. Verify that the DATASEC in the header above applies to all the frames by using the fitsheader command:

> fitsheader --keyword DATASEC m92\*.fits

Copy & paste the output below. Now do the same to verify that all the BIASSEC values are the same in each file.



Now, let's trim the overscan region from all our images. The syntax for trimming the data array (remember, this is still just our master bias image from above) to keep all of the first index but only the first N rows of the second index is:

In [ ]:
#Replace N with the correct value, corresponding to (no. columns in the DATASEC above) -1 (zero indexing!)
masterbias_trimmed = data[:,:N]
print(masterbias_trimmed.shape)

Refer to the Python tutorial, specifically the Numpy section on indexing and slicing for more 
information: <br>
http://www.scipy-lectures.org/intro/numpy/array_object.html#indexing-and-slicing

Now, write this new trimmed image to a fits file. (This step isn't always necessary. If you're just going to move on to futher reduction steps that use this trimmed array, you might not need to write out this array to a fits file.)

In [ ]:
hdu = fits.PrimaryHDU(masterbias_trimmed)
hdu.writeto('trim_m92001.fits')

This subtraction of the overscan needs to be done to all the other FITS files in this directory by starting with the appropriate steps above to read the FITS data into an array and trim the array as in the previous step. 

Let's devise a quick way to do this. In this directory where your ccd1.ipynb file and all your data files are located, make a list called "files.list" of all the files *excluding* the master bias frame. 


In [ ]:
dt=np.dtype([('filename', np.unicode, 20)])
files=np.genfromtxt("files.list",dtype=dt)
infiles=files['filename']

for i in range (0,len(infiles)):
    sci_fn = infiles[i]
    out= "trim_"+infiles[i]
    sci_hdulist = fits.open(sci_fn)
    data = sci_hdulist[0].data.astype(np.float)
    trimmed=data[:,:320]
    hdu = fits.PrimaryHDU(trimmed)
    hdu.writeto(out)
    print ("writing ", out)

Verify that you now have a set of FITS files called trim\_\*.fits corresponding to each file you started out with.

## Bias subtraction

The next step in the reduction process is to remove the bias level from all frames. The bias level is contained in the your trimmed master bias frame, here called trim\_m92001.fits 


In [ ]:
sci_fn = 'trim_m92001.fits'
sci_hdulist = fits.open(sci_fn)
bias = sci_hdulist[0].data.astype(np.float)

for i in range (0,len(infiles)):
    sci_fn = "trim_"+infiles[i]
    out= "bias_"+infiles[i]
    sci_hdulist = fits.open(sci_fn)
    data = sci_hdulist[0].data.astype(np.float)
    biassub=data-bias
    hdu = fits.PrimaryHDU(biassub)
    hdu.writeto(out)


Verify that you now have a set of files called bias\_m92006.fits, bias\_m92007.fits, etc. These are the frames that have been both trimmed and bias subtracted.

**QUESTION:** Calculate the image statistics (mean, median, standard deviation, min & max) on: (1) the master bias frame; (2) the pre-bias-corrected frames; and (3) the post-bias-corrected frames and note them here. Are these values consistent with your expectations?


## Flat field

We have finally arrived at the flat fielding stage. We have two flats and they need to be normalized before we divide our object frames by *the flat field frame appropriate for its filter*.

We will use image statistics to determine the normalization value for each flat, and then use our same image math techniques to create the normalized flats.

You now have all the tools to perform these tasks. Here are the steps:

 1. Determine which frame is the flat in the B filter and which is the flat in the V filter by examinine the header information. Usually that is in a FILTER header keyword. Here the information you need is in the OBJECT keyword:

 > fitsheader --keyword OBJECT m92\*.fits

 2. Find the mode of the B and V flats:

In [ ]:
#Replace NN with the correct number for the B flat field image
from scipy import stats

sci_fn = "bias_m92014.fits"
sci_hdulist = fits.open(sci_fn)
Bflat = sci_hdulist[0].data.astype(np.float)
stats.mode(Bflat, axis=None)

 3.Normalize the flat field by its mode


In [ ]:
#Replace MODE with the value of the previous operation.
nBflat=Bflat/MODE
nBflat[nBflat == 0] = 1.  
#This last step replaces any pixels that are =0 with a value of 1 in the normalized flat. This
#is necessary to avoid errors when we divide by this frame below.

#Add extra cells to repeat the previous step and this step for the flat in the V filter.
#Call the files Vflat and nVflat.

**QUESTION:** What effect does this division by the mode have on the image, i.e. why do we refer to this as normalizing the flat field frames?

**QUESTION:** What are the statistics (mean, median, standard deviation) for these normalized flat fields? Comment on the values of the mean & median for a normalized image.

4.Now divide each of the object frames by the appropriate normalized flat for that filter. And write to a new fits file. Add extra cells for the additional data frames as necessary.

In [ ]:
#Replace NN as two cells above
sci_fn = "bias_m920NN.fits"
sci_hdulist = fits.open(sci_fn)
data = sci_hdulist[0].data.astype(np.float)
fdata= data/nBflat #or nVflat
hdu = fits.PrimaryHDU(fdata)
hdu.writeto('flat_m920NN.fits')

Look at these final images with ds9. Check to see if the background is flat across the image.  Sometimes the dome flats are not sufficient for flattening images and additional sky flats may need to be used. For your observations, you'll just use these sky flats, which can be normalized in the same way.

You now have a set of images that have been zero and flat corrected. As noted above, you will not have to do an overscan substraction for the data frames for your observing project. You will however, need to do a dark current subtraction after averaging your dark current frames and scaling them by the exposure time in your data frames.

Write the commands for flat field correction into your ccd1\_reduce.py script for later use. You will not necessarily need to use a for loop for these, due to the small number of frames involved in each calculation, but depending on the number of files you have to reduce for your project, you may want to edit this later. 

Often in data reduction and analysis, there are many choices of methods that all give the correct output, just choose what makes the most sense for you. It is good practice to devise ways to check your results by calculating image statistics or using a viewer like ds9.